## 네이버 영화평 감성분석

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

- 데이터 가져오기:'네이버 영화리뷰 데이터'로 검색

In [3]:
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",sep='\t')
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

In [5]:
train_df.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


#### 데이터 전처리

- 트레인 데이터 셋

In [6]:
# 중복 여부 확인
train_df.document.nunique()

146182

In [8]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

In [10]:
# Null 데이터가 있는지 확인
train_df.isna().sum()

id          0
document    1
label       0
dtype: int64

In [12]:
# Null 데이터 제거
train_df = train_df.dropna(how='any')
train_df.shape

(146182, 3)

In [14]:
# 부정, 긍정 데이터 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터 셋

In [15]:
#중복 여부 확인
test_df.document.nunique()

49157

In [16]:
# 중복 데이터 제거
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49158, 3)

In [17]:
test_df.isna().sum()

id          0
document    1
label       0
dtype: int64

In [18]:
test_df = test_df.dropna(how='any')
test_df.shape

(49157, 3)

#### 2. 텍스트  전처리

- Train dataset

In [20]:
# 한글과 공백 이외의 문자는 제거

train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [22]:
# 데이터가 다 지워져서 ''가 된 데이터 --> np.nan 으로 변경한 후 제거
train_df.document.replace('',np.nan, inplace=True)
train_df.isna().sum()

id            0
document    391
label         0
dtype: int64

In [23]:
train_df = train_df.dropna(how='any')
train_df.shape

(145791, 3)

- Test dataset

In [24]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
test_df.head(3)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0


In [25]:
test_df.document.replace('',np.nan, inplace=True)
test_df.isna().sum()

id            0
document    162
label         0
dtype: int64

In [26]:
test_df.head(3)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,NaN,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0


In [27]:
test_df = test_df.dropna(how='any')
test_df.shape

(48995, 3)

- 전처리가 끝난 데이터는 저장

In [28]:
train_df.to_csv('data/naver_movie_train.tsv', sep='\t', index=False)
test_df.to_csv('data/naver_movie_test.tsv', sep='\t', index=False)

#### 3. 한글 처리

In [29]:
from konlpy.tag import Okt
okt = Okt()

In [30]:
text = '흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나'
okt.morphs(text)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍지', '않구나']

In [33]:
okt.morphs(text, stem=True)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']

In [34]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [35]:
stop_text = '은 는 이 가 을 를 의 에게 에 들 좀 잘 과 도 으로 자 와 ㅋㅋ ㅠㅠ ㅎㅎ'
stopwords= stop_text.split()

In [38]:
from tqdm.notebook import tqdm
X_train = []
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence,stem=True)
    tmp_list = [word for word in morphs if word not in stopwords]
    tmp_str = ' '.join(tmp_list)
    X_train.append(tmp_str)

  0%|          | 0/145791 [00:00<?, ?it/s]

In [39]:
X_test = []
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence,stem=True)
    tmp_str = ' '.join( [word for word in morphs if word not in stopwords])
    X_test.append(tmp_str)

  0%|          | 0/48995 [00:00<?, ?it/s]

In [40]:
y_train = train_df.label.values
y_test = test_df.label.values

#### 4. Feature 변환

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer()
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)
X_train_cv.shape, X_test_cv.shape

((145791, 42115), (48995, 42115))

#### 5.모델 학습/평가

In [42]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_cv, y_train)
lr.score(X_test_cv, y_test)

0.8251862434942341

In [56]:
review1 = '객 오랜 스파이더맨 팬들을 위한 선물이다. 20년간의 모든 이야기'
review2 = '유치하고 암걸리는 OOO기 영화. 마블영화중에서 재미있는건 하나도 없음. 그나마 CG가 멋져서 보는애들 많은데 내 기준에서는 저능아들이나 좋아할 영화임. 영화비 12000원으로 차에 기름넣는게 더 알차다.'


#### 6. 실제 테스트

In [57]:
# 전처리
import re

review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',review1)
review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',review2)
review2

'유치하고 암걸리는 기 영화 마블영화중에서 재미있는건 하나도 없음 그나마 가 멋져서 보는애들 많은데 내 기준에서는 저능아들이나 좋아할 영화임 영화비 원으로 차에 기름넣는게 더 알차다'

In [58]:
# 불용어 처리
morphs = okt.morphs(review1, stem=True)
review1 = ' '.join(word for word in morphs if word not in stopwords)
review1

'객 오랜 스파이더맨 팬 위 한 선물 이다 년 간 모든 이야기'

In [50]:
review2 = ' '.join(word for word in okt.morphs(review2, stem=True) if word not in stopwords)
review2

'유치하다 암 걸리다 기 영화 마블 영화 중 에서 재미있다 하나 없다 그나마 가다 멋지다 보다 애 많다 내 기준 에서는 저능 아들 이나 좋아하다 영화 임 영 화비 원 차 기름 넣다 더 알차다'

In [59]:
# Feature 변환
review1_cv = cvect.transform([review1])

In [60]:
lr.predict(review1_cv)

array([1], dtype=int64)

In [55]:
review2_cv = cvect.transform([review2])
lr.predict(review2_cv)

array([0], dtype=int64)